# Replication for Williams, K. R. (2022). The welfare effects of dynamic pricing: Evidence from airline markets

Structural Econometrics ECON-L6210

Fall 2023

Replication report

Helena Rantakaulio, Hung Le, Babak Firoozi Fooladi

## Introduction

We attempted to replicate the main results for the paper Williams, K. R. (2022). The welfare effects of dynamic pricing: Evidence from airline markets. Econometrica, 90(2), 831-858. The original replication package can be downloaded from https://www.econometricsociety.org/publications/econometrica/2022/03/01/welfare-effects-dynamic-pricing-evidence-airline-markets and the repository for the code of the original replication package is https://github.com/kw468/welfare-airlines.

The original replication package uses a proprietary package Knitro for optimization and requires gpu-enabled Jax. We did not have access to those so we were required to rewrite some parts of the code. In order to fully understand how the code works we ended up rewriting most of it in Julia whereas the original replication package is written in python.

We think we managed to write the parts of the code that are needed to generate the main results up until the optimization algorithm for the likelihood. But we were unable to replicate the results as the optimization algorithm took a very long time to run and we ran out of time. We compared the output of the programs written by us to the output of the corresponding programs in the replication package, excluding the final optimization algorithm, and are convinced that the scripts produce the same output as the replication package.

Original packages contains several scripts for data preparation, plots, tables robustness checks and bash scripts for automation and execution. Our aim only requires replication of following scrips in the original repository:

| Usage                                                                                                                      | Script                                               |
|----------------------------------------------------------------------------------------------------------------------------|------------------------------------------------------|
| Data preparation                                                                                                           | scripts/estimation/estim_model_jax_multiFE_EF_sms.py |
| Likelihood function - demand calculation - expected return - expected value - transition matrix - dynamic estimation - CCP | scripts/setup/estimation_setup.py                    |


## Replication functions

### Required packages

In [1]:
using DataFrames, JuMP
using Statistics, Clustering, StatsBase, SpecialFunctions
using Query, FreqTables
using ShiftedArrays
using Dates, Missings, Random
import Parquet2, CSV
using Optim # for BFGS -> KN_HESSOPT_BFGS
using MKL # Using Julia with Intel's MKL -> KN_BLASOPTION_INTEL
using NLopt # Non-Linear Optimization package

### Calculating the probability of a consumer being business type

There are two types of consumers in the model, leisure travelers with a lower willigness to pay and business travelers with a higher willingness to pay. A consumer is a business traveler at time $t$ with probability $\gamma_t$ and a leisure traveler with probability $1-\gamma_t$. $t=0$ denotes the first sales period of tickest to a flight and $t=T$ corresponds to the flight departure date. The data for each flight runs for 60 days before the departure, so $T=60$.

The authors assume that probability of being business type is determined byt the equation below. This specificaiton keeps probability between 0 and 1 and allows for non-monotonicity in consumer types over time.
Parameters $\gamma_0$, $\gamma_1$, and $\gamma_2$, are estimated in the model. Parameter $T$ is set to 60 days

$$Pr_{t}(\text{Business}) =  \gamma_{t}=\frac{\exp{(\gamma_{0} + \gamma_{1} t+\gamma_{2} t^{2}})}{1+\exp{(\gamma_{0}+ \gamma_{1} t+ \gamma_{2} t^{2})}} = \frac{1}{1+\exp{(-\gamma_{0} - \gamma_{1} t - \gamma_{2} t^{2})}}$$


The following function returns the vector of probabilities for all $t=0,1,...,T$

In [2]:
PR_B(γ_0, γ_1,γ_2, T) = 1 ./ (exp.(-γ_0 .- (0:(T-1)) .* γ_1 .- ((0:(T-1)) .^ 2) .* γ_2) .+ 1)

PR_B (generic function with 1 method)

### Calculating choice probabilities

As noted in the paper, with IID T1EV distributed idiosyncratic preferences the individual choice probabilities equal
$$
\pi_{j t}^{i}(p_{j t})=\frac{\exp(x_{j t}\beta_{i}-\alpha_{i}p_{j t})}{1+\exp(x_{j t}\beta_{i}-\alpha_{i}p_{j t})} = \frac{1}{1+\exp(- x_{j t}\beta_{i} + \alpha_{i}p_{j t})}
$$

with $\gamma_{t}$ share of consumers being business type, integrating over consumer types the product shares are equal to

$$
\pi_{j t}(p_{j t})= \gamma_{t}\,\pi_{j t}^{B}({\mathcal{P}}_{j t})+\left(1 - \gamma_{t}\right)\pi_{j t}^{L}({\mathcal{P}}_{j t})
$$

After calculating $\pi_{j t}(p_{j t})$, we proceed with demand probability. The authors assume consumers arrive according to Poissin distribution $\tilde{M}_t \sim \text{Poisson}_t(\mu_t)$. Then, demand is also distributed Poisson, $Q_{jt} \sim \text{Poisson}_t(\mu_t \pi_{jt})$. We get the probability that q seats are demanded on flight j at time t:
$$
\underset{t} {Pr}(Q_{j t} = q; p_{j t}) = \frac{\left(\mu_{t} \pi_{j t} \right)^{q} \exp \left(-\mu_{t} \pi_{j t}\right)}{q!}
$$

So, the expected sales with demand censored at capacity equals 
$$
    Q_{jt}^e (p_{jt};c_{jt})= \sum_{q=0}^{c_{jt}-1} Pr_t(Q_{jt}=q;p_{jt})q + \sum_{q=c_{jt}}^{\infty} Pr_t(Q_{jt}=q;p_{j t}) c_{jt}
$$

$$
\log\bigg(\underset{t} {Pr}(Q_{j t} = q; p_{j t})\bigg) = q \log\left(\mu_{t} \pi_{j t} \right) -  \left(\mu_{t} \pi_{jt}\right) - log(q!)
$$


In [3]:
function log_demandQ_tγ(β, bL, bB, γt, μt, q, Pt)
    """ The log
        input:
            γt: Consumer type prob at time t 1x1
            μt: mean demand at time t 7x1
            q : seat count 1x1
            β : demand parameters 7x1
            bL: Leisure demand parameters 1x1
            bB: Business demand parameters 1x1

        output: 
            log of demand probability matrix 8x7
    """
    #* Product share:

    # Probability of Leisure booking on time t
    πL_t = (1 .-γt) ./(1 .+ exp.(-β' .- bL .* Pt))
    # Probability of Business booking on time t
    πB_t = (γt) ./(1 .+ exp.(-β' .- bB .* Pt))

    π_t = πL_t .+ πB_t
    # --------------------------------------------------------------------------

    #* Log of Demand probability:

    # demand calculation for time t, seat count q
    Q_jt = q .* (log.(π_t) .+ log.(μt)') .- (π_t'.*μt)'  .- loggamma(q+1)

    return Q_jt
end

#* Demand probability:
# for Q == q at time t
demandQ_tγ(β, bL, bB, γt, μt, q, Pt) = exp.(log_demandQ_tγ(β, bL, bB, γt, μt, q, Pt))

# The demand for all T for Δq is calculated given the parameters 
Demand_q_T(β, bL, bB, γ, μ, Δq, Pt) = [demandQ_tγ(β, bL, bB, γt, μ[t,:], Δq, Pt) for (t,γt) in enumerate(γ)];

### Capacity Transition matrix $f(\cdot)$

This matrix is used to obtain the transition probabilities values:
$$
h_{t} (\omega_{t+1}, c_{t+1} | \omega_{t}, c_{t}, p_{t}) = g(\omega_{t+1}) f_{t} (c_{t+1} | p_{t}, c_{t})
$$

And it is using the demand function in previous blocks.

Generation of this matrix eliminated the need for repetitive calculation of the probabilities.
Moreover, this matrix is later used to expected revenue and expected future value function.



In [4]:
function create_f_mat(β, bL, bB, γ, μ, q̄, Pt)
    """
    input:
        T: Max number of days before departure
        t: time ∈ {,1,...,T}
        q̄: maximum seat count
        Δq: change in seats ∈ {0,1,..., q̄}
        Pt: clustered prices
        γ: probability of type Business
        μ[t,:] : Arrival rates (to be estimated)
        bL: Leisure demand parameter (to be estimated)
        bB: Business demand parameter (to be estimated)
        β : demand parameters (to be estimated)
    output:
        f: transition matrix 
    """
    Prob_sellout = []
    for Δq in 0:(q̄)
        # demand matrix is used here:
        temp = Demand_q_T(β, bL, bB, γ, μ, Δq, Pt) .|>
            x -> x .|>
            y -> ifelse.((y < 1e-100) & (y != 0), 1e-100, y) 
            
        push!(Prob_sellout, temp)
    end

    zero_PT_β = zeros(length(Pt),length(β))
    f = [[[zero_PT_β for i in 1:T] for _ in 1:(q̄+1)]]
    for Δq in 1:(q̄)
        temp = deepcopy(Prob_sellout[1:Δq])
        push!(temp, sum(temp[1:Δq]) .|> x -> 1 .- x)
        for q in 1:(q̄-Δq)
            push!(temp, [zero_PT_β for i in 1:T])
        end
        push!(f,temp)
    end

    return f
end



create_Expected_R (generic function with 1 method)

### Expected Revenue

We proceed with creating expected revenues matrix for each price at every time $P_{t}$ for remaining seats $c_{t}$.
$$
R^{e}_{t}(P_{t};c_{t}) = p_{t} \cdot Q^{e}_{t}(P_{t};c_{t})
$$

Where $Q^{e}_{t}(P_{t};c_{t})$ is function in previous blocks.

In [ ]:
function create_Expected_R(f, q̄, Pt)
    """
    input:
        f: transition matrix 
        q̄: maximum seat count
        Pt: clustered prices
    output:
        ER: Expected revenue matrix
    """
    Pt_T = ((0:(q̄+1)) .* Pt')
    ER = [sum([f[q1][q] .|> x ->  x .* Pt_T[q,:] for q in 1:q1]) for q1 in 1:(q̄+1)];
    return ER
end

The function for expected value is:
$$\mathrm{EV}_{t}(p_{t},c_{t})=\int_{c_{t+1}}\Biggl[\sigma\ln\Biggl(\sum_{p_{t+1}\epsilon\lambda(t+1)}\exp\Biggl(\frac{R_{t+1}^{e}(c_{t+1},\,p_{t+1})+\mathrm{EV}_{t+1}(p_{t+1},c_{t+1})}{\sigma}\Biggr)\Biggr)\Biggr] \times\,f_{t}(c_{t+1}|c_{t},\,p_{t})+\,\sigma\phi,$$

To calculate this dynamic process, the code sets the value for the day after departure as zero. Therefore we can calculate the values from time $t = T$ recursively to $t = 0$. This is done in the function below.

The function below also calculates the CCP.


$$
CCP_t(p_t,c_t)= \frac{exp[R_{t}^e (c_{t},p_{t})+ EV_t(p_t,c_t)/\sigma]}{\sum_{p'_t\in A(t)} exp[R_{t}^e (c_{t},p'_{t})+ EV_t(p'_t,c_t)/\sigma]}
$$




In [5]:
# Helping functions for DRY:
gpr_qt(ER, EV, Pt_m, t, σ) = ((ER + EV) ./ σ) .* Pt_m[t, 2:end]
V_T(grp_q, EC, σ) = σ * (log.(sum(exp.(grp_q), dims=1)) .+ EC)
inf_to_zero(x) = ifelse.(x .== -Inf, 0, x);


# Dynamic demand estimation
function dynEst(f, ER , σ, T, Pt_m ,q̄, β)

    # generating the EV at t = T, where EV(T+1) = 0
    EV = Dict()
    V = Dict()
    CCP = Dict()
    t = T
    grp = []
    EC = 0.5772156649 # Euler constant
    EV[t + 1] = [zeros(length(Pt),7) for i in 1:(q̄+1)]

    for q in 1:(q̄+1)
        push!(grp, gpr_qt(ER[q][T], EV[T+1][q] , Pt_m, t, σ))
    end

    grp = grp .|> x -> ifelse.(x .== 0, -Inf, x);
    V[t] = V_T.(grp, EC, σ);
    CCP[t] = grp .|> x -> x .- log.(sum(exp.(x), dims=1));
    replace!.(CCP[t],NaN => 0.0)
    

    # Recursion for t = T,...,1
    for t in (T):-1:2
        grp = []
        for q in 1:(q̄+1)
            push!(grp, gpr_qt(ER[q][t], EV[t+1][q] , Pt_m , t, σ))
        end
        grp = grp .|> x-> ifelse.(x .== 0, -Inf, x);
        CCP[t] = grp .|> x -> x .- log.(sum(exp.(x), dims=1));
        replace!.(CCP[t],NaN => 0.0)

        V[t] = V_T.(grp, EC, σ);

        EV[t] = []
        for q in 1:(q̄+1)
            g_q = f[q] .|> x -> x[t]
            V_q = collect(vcat([zeros(1,7)], V[t][1:(length(g_q))]) )
        
            for q_r in 1:q
                push!(EV[t], g_q[q_r] .* V_q[q_r])
            end
        end
        replace!.(CCP[t],-Inf => 0.0)
    end
    
    return CCP
end



dynEst (generic function with 1 method)

## Log-Likelihood Function

In following block Log-Likelihood is coded. It calculates the transition matrix, Expected Revenue and Expected Value in itself for dynamic estimation. 

$$
\underset{(\beta, \alpha, \gamma_{t}, \mu_{t}, \sigma)} \max \sum_{F} \sum_{T} \log \big(\text{CCP}_{t} ( p_{t}; c_{t})\big) + \log \big(f_{t}(c_{t+1} | c_{t}, p_{t})  \big)
$$

In [6]:

function logLike(X₀, data, T, q̄, Pt)
    # Variables specs:
    """
    T: Max number of days before departure
    t: time ∈ {,1,...,T}
    q̄: maximum seat count
    Δq: change in seats ∈ {0,1,..., q̄}
    Pt: clustered prices
    γ: probability of type Business
    μ[t,:] : Arrival rates (to be estimated)
    bL: Leisure demand parameter (to be estimated)
    bB: Business demand parameter (to be estimated)
    β : demand parameters (to be estimated)
    """

    EC =  0.5772156649 # Euler constant
    γ = PR_B(X₀[10], X₀[11], X₀[12], T);
    μ₁ = X₀[13]
    μ₂ = X₀[14]
    μ₃ = X₀[15]
    μ₄ = X₀[16]

    μT = [μ₁ .* ones(T - 20); μ₂ .* ones(7); μ₃ .* ones(7); μ₄ .* ones(6)]
    μ_DoW = [1; X₀[17:22]]

    μ = μT * μ_DoW';

    β = X₀[1:7]
    bL = min(X₀[8], X₀[9])
    bB = max(X₀[8], X₀[9])

    σ = X₀[end];

    EC = 0.5772156649 # Euler constant
    β = X₀[1:7]
    
    f = create_f_mat(β, bL, bB, γ, μ, q̄, Pt)
    ER0 = create_Expected_R(f, q̄, Pt)
    
    loss0 = []
    for x in 1:size(data)[1]
        push!(loss0, f[data[x,1]][data[x,2]+1][data[x,3]+1][data[x,4] + 1, data[x,5] + 1])
    end
    loss0 = ifelse.(loss0 .== 0, 1, loss0)
    loss0 = log.(loss0);
    loss0 = sum(loss0)
    CCP0 = dynEst(f, ER0 , σ, T, Pt_m ,q̄, β)
    
    loss1 = []
    for x in 1:size(data)[1]
        
        push!(loss1, CCP0[data[x,3]+1][data[x,1]+1][data[x,4]+1,data[x,5]+1])
        
    end

    loss1 = sum(loss1)

    return loss0 + loss1
end



logLike (generic function with 1 method)

## Setting up the dataset

The author clusters all observed prices of a given route using a k-means algorithm with minimum in-smaple threshold of 99%. This is done to avoid constructing the likelihood for each flight individually, so practically they want to reduce the computation time. The resulting pricing choice sets range in size from 5 to 11. We used the original code to produce the prices and planned to use them in our model, in order to be able to replicate the paper's results.

Moreover, because of the difference, the observations values in `Pt_m` are different in our model compared to the original estimation. Therefore we use the output of the original replication data. 
What does this mean?

We try to decrease the computational burden by using only a single route for the estimation. We tried this with the route `"BOS_SAN"`.  

### Replication dataset preparation (for changing the route):
For other routes script `Generate data.jl` will produce the necessary values. 

In [4]:
include("Scripts/Generate data.jl") # this script creates the panel for each route

bndsLo = [
        -10, -10, -10, -10, -10, -10, -10, -10, -10,
        -250, -10, -.06, .1, .1, .1, .1,
        .01, .01, .01, .01, .01, .01, .02
    ];

bndsUp = [
    15, 15, 15, 15, 15, 15, 15, 0, 0, 40, 10, .15,
    15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 2
];

X₀ = [ # Initial values
    2.49999999, 2.49999999, 2.49999999, 2.49999999, 2.49999999, 
    2.49999999, 2.49999999, -1.05185291, -0.72189149, -13.39650409, 
    0.27373386, 0.0, 1.91183252, 2.46138227, 1.82139054, 2.35728083, 
    1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.22463165
];

df = DataFrame(CSV.File("data/original_paper/BOS_SAN_data.csv"));
data = convert.(Int,Matrix(df));

Pt_m = DataFrame(CSV.File("data/original_paper/BOS_SAN_Pt.csv"));
Pt_m = Matrix(Pt_m)

Pt = [2.07262418, 2.86435267, 3.38673056, 3.99072046, 4.66267888, 5.55337481, 6.40260199, 7.38572223];

T = 60
q̄ = maximum(data[:,1])

nothing

### Optimization

This block uses NLopt.jl for non-linear optimization of the model to estimate the parameters that maximize the likelihood function.

**NOTE:** We are almost certain that this code snippet would do the optimization. However, we were unable to complete this step as the estimation took way too long and we ran out of time.

In [8]:
using NLopt
opt = Opt(:LD_MMA, 23) # Choose the algorithm and the number of parameters
lower_bounds!(opt, bndsLo)
upper_bounds!(opt, bndsUp)

# Define the objective function
max_objective!(opt, (x, grad) -> logLike(x, data[1:100,:], T, q̄, Pt))


# Run the optimization
(minf,minx,ret) = NLopt.optimize(opt, X₀)

## Evaluation of the replication package

There are some clear pros to the replication package. Most notably, the ReadMe files give goes instructions on how to use the replication package and the code files include clear descriptions and comments on how the code works. The author also included the estimation results in the replication package, so basically we would be able to replicate the tables in the paper using just the estimation results given.

There are also some clear cons. First, the replication package requires a proprietary package Knitro for optimization and gpu-enabled Jax. These are not very common at least for economists so if you don't have access to those resources you'll have to do some extra work. Second, it seems that some parts of the code were written in an overly complicated way which made the code difficult to follow regardless of the commenting. Nevertheless, it is understandable as these complications are the results of converting calculations to numerical operations that could benefit from GPU hardware capabilities.